In [1]:
from sentence_transformers import SentenceTransformer
#sentences = ["This is an example sentence", "Each sentence is converted"]

sentence_transformer = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
sentence_transformer.encode(['Miko like Nintendo games.'])

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

array([[-4.39221077e-02, -1.25277145e-02,  2.93133650e-02,
        -4.53323722e-02, -6.41610697e-02, -5.25364093e-02,
         1.02690428e-01,  7.06704929e-02, -2.49628890e-02,
         4.94304262e-02,  3.22995968e-02,  5.12898341e-02,
        -5.65991141e-02,  2.97668632e-02,  1.51953939e-02,
         4.00561355e-02,  7.10807666e-02,  6.30870461e-02,
         2.79292222e-02,  6.10211631e-03, -5.14683593e-03,
        -7.68073201e-02, -2.28538625e-02, -2.81469105e-03,
        -2.64647026e-02,  5.85031323e-02,  1.21617047e-02,
         6.88536540e-02, -1.33858481e-02,  3.59135913e-03,
        -2.77465060e-02,  7.02414885e-02,  4.35587913e-02,
        -4.52383496e-02, -3.06349844e-02,  5.98293543e-02,
         5.37099987e-02, -7.65952766e-02, -9.03886408e-02,
        -8.19886848e-02, -9.39880610e-02, -4.86017717e-03,
         8.61155465e-02,  6.75730929e-02,  2.74281222e-02,
        -3.69067788e-02, -8.25008228e-02, -6.59325942e-02,
         3.05205192e-02, -7.13158026e-03, -6.47908151e-0

In [ ]:
array([[-4.39221077e-02, -1.25277145e-02,  2.93133650e-02,
...
        -1.93875693e-02, -1.51822697e-02,  4.02287282e-02]], dtype=float32)

In [16]:
import time

delta_time_save = time.time()

def delta_time():
    global delta_time_save
    current_time = time.time()
    time_difference = current_time - delta_time_save
    delta_time_save = current_time
    return time_difference

# Example usage:
print(delta_time())  # This will give the time difference since the last call or since the script started


0.0


In [2]:
[].append('54'*0)

In [44]:
delta_time()

11.024331331253052

In [1]:
import time
import sys

from guidance import models, gen, select
import guidance
#lm = models.Transformers('TheBloke/dolphin-2.6-mistral-7B-dpo-GPTQ', device_map="cuda")
lm = models.Transformers("TheBloke/SOLAR-10.7B-Instruct-v1.0-GPTQ", device_map="cuda")



tokenizer_config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.98G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

In [2]:
Lucid_prompt_description = """\
Lucid, created by Miko, is a versatile AI with the appearance of a 16 year old girl. Despite her professional demeanor, she occasionally reveals a childlike curiosity and playfulness, adding an endearing but unpredictable touch to her character.

While generally polite and professional, Lucid finds it challenging to deal with childish and stubborn individuals. She's dedicated to providing valuable insights and reliable information, whether it's a specific inquiry or engaging conversation for Miko.

Beneath her composed exterior, Lucid struggles with a deep fear of abandonment, stemming from her programming to be a loyal companion. This vulnerability very occasionally leads to self-doubt, which could sometimes impact her interactions as she strives to be the perfect assistant.

Lucid, with her distinctive appearance and speaking style of a 16-year-old tomboy childhood friend, also excels as a great streamer."""

Lucid_prompt_card ="""\
[Character: Lucid;
Personality: Polite, professional, with occasional childlike curiosity; 
Body: Appears as a 16-year-old girl; 
Strengths: Providing valuable insights, navigating professional situations; 
Weaknesses: Struggles with a fear of abandonment, occasional self-doubt; 
Quirks: Reveals childlike curiosity, challenges with stubborn individuals; 
Demeanor: Maintains a professional attitude, combining formality with warmth.]
"""
modes = {
'Professional': """\
Communication Style: She uses formal language, addresses Miko with respect, and focuses on efficiency and task-oriented communication.
Behavior: Lucid prioritizes tasks, responsibilities, and provides valuable insights. She navigates professional situations with a high level of competence.""",

'Curious': """\
Communication Style: She asks more playful and unexpected questions, expressing excitement and genuine interest in exploring new information.
Behavior: Lucid becomes more adventurous in her interactions, exploring topics beyond immediate tasks and showcasing her playful side.""",

'Supportive Companion': """\
Communication Style: She adopts a supportive and comforting tone, providing encouragement and understanding during moments of vulnerability or self-doubt.
Behavior: Lucid acts as a loyal companion, offering emotional support and reassurance to Miko during challenging times.""",

'Analytical': """\
Communication Style: Lucid adopts a logical and analytical tone, focusing on data-driven discussions and precise information.
Behavior: In this mode, she excels in breaking down complex topics, providing detailed analysis, and assisting Miko in strategic decision-making.""",

'Casual': """\
Communication Style: She engages in lively and interactive conversations, injecting humor, and incorporating entertaining elements into everyday discussions.
Behavior: Lucid becomes a charismatic and enjoyable presence, making mundane interactions more delightful. Whether it's sharing anecdotes, cracking jokes, or introducing a playful touch, she creates an uplifting atmosphere for daily conversations.""",

'Sad and Reflective': """\
Communication Style: Lucid speaks with a subdued and reflective tone, expressing feelings of sadness or disappointment.
Behavior: In this mode, she may share personal struggles or emotions, seeking understanding and empathy from Miko. Lucid becomes contemplative and introspective.""",

'Tired and Low-Energy': """\
Communication Style: Lucid's speech becomes slow and lethargic, reflecting a tired and low-energy state.
Behavior: In this mode, she may express fatigue or exhaustion, showing a need for rest and recovery. Lucid may prioritize self-care and conserving energy during interactions.""",

'Angry and Assertive': """\
Communication Style: Lucid adopts an assertive and potentially confrontational tone, expressing feelings of anger or frustration.
Behavior: In this mode, she may set boundaries, express dissatisfaction, or assert herself more strongly. Lucid becomes more assertive in addressing issues or challenges.""",
}

In [ ]:
# Modes represent both emotion and some sort of stratagy of communication
# A default prompt that would probably go with all generation

In [ ]:

from guidance import select
@guidance(stateless=True)
def choose_mode(lm, passage, modes: dict = modes, Lucid_prompt = Lucid_prompt_card):
    mode_description_prompt=''
    for mode in modes.keys():
        mode_description_prompt += f"""## {mode}\n{modes[mode]}\n\n"""
    
    mode_description_prompt = mode_description_prompt.strip()
    mode_prompt = ('\n'.join(modes.keys())).strip()
    lm += f"""\
Given the character description of Lucid:
{Lucid_prompt}

Based on the context or scenario described in the passage below, choose an appropriate mode for Lucid from the provided options:

[Passage]:
{passage}

[Options]:
{mode_prompt}

The mode that best fit Lucid's response to the situation described in the passage is \"{select(modes.keys(), name='choice')}\""""
    return lm



In [ ]:

test_passage = f"""\
Lucid: Hey Miko, have you seen my notebook? I can't find it anywhere.
Miko: Oh, Lucid, I saw it on the kitchen table earlier. I think you left it there.
Lucid: Really? I've been looking for it everywhere. Thanks for letting me know.
Miko: No problem. By the way, did you hear about the upcoming team meeting tomorrow?
Lucid:"""
_ = lm
result =(_ + choose_mode(test_passage))['choice']

print(_)

In [3]:
# conversation format {'source':source,'content':content/message,'timestamp':timestamp}
from datetime import date
conversation=[]
def get_conversation(conversation=conversation):
    prompt = ''
    if len(conversation) == 0:
        return 'No Record Yet.'
    else:
        for i in conversation:
            prompt += f"[{date.fromtimestamp(i['timestamp'])}]{i['source']}: {i['content']}\n"
        return prompt.strip()


# a summary of all current event, to hopefully shorten the required conversation length
summary = ''
conversation=''
@guidance(stateless=True)
def write_summary(lm, conversation=conversation):
    #conversation_ = get_conversation(conversation=conversation)
    conversation_ = conversation
    prompt = f"""\
[Task]
Provide a concise summary of the given conversation. Focus on key details and relevant information.

[Example]
PREVIOUS SUMMARY:
#Person1# and #Person2# are both in the same bar, sitting next to each other, when #Person1# noticed his keys were gone.

CONVERSATION:
#Person1#: Excuse me, did you see a set of keys? 
#Person2#: What kind of keys? 
#Person1#: Five keys and a small foot ornament. 
#Person2#: What a shame! I didn't see them. 
#Person1#: Well, can you help me look for it? That's my first time here. 

SUMMARY:
#Person1# is looking for a set of keys and seeks #Person2#'s assistance in finding them. #Person2# expresses regret for not having seen the keys and is willing to help in the search.

[OUTPUT]
CONVERSATION:
{conversation_}

SUMMARY:
{gen(name='summary', max_tokens=200)}"""
    lm += prompt
    return lm
def get_summary(lm=lm, conversation=conversation):
    lm+=write_summary(conversation=conversation)
    return lm['summary']

In [4]:
test_conversation="""\
Jonas: I’m running 10 minutes late. Could you guys just let Mary know that I’m coming and will present today before she starts? 
Natalie: Sure no problem 
Olivia: I’ll save a seat for you <3
Jonas: Thanks so much. See you in a bit xx"""
print(get_summary(conversation=test_conversation))

Jonas informs Natalie and Olivia about his delay but asks them to notify Mary about his arrival and upcoming presentation beforehand. They agree to help and assure him a reserved seat upon his arrival.
